In [1]:
#Imports
import pandas as pd
import numpy as np
import plotly.express as px
import random
import plotly.io as pio
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [2]:
#Function that calculates the value of the sigmoid function
def sigmoid(z):
    return 1.0/(1 + np.exp(-z))

In [3]:
#Function that predicts the value of the class using a single-layer neural network
def predict(X,weights,bias):
    # X --> Input vector
    
    # Calculating the value of y using the inner-product (dot product) of the weights vecotr with the data vector and adding a bias term
    y = np.dot(X, weights) + bias
    preds = sigmoid(y)
    
    # Empty List to store predictions.
    pred_class = []
    for i in range(0,len(preds),1):
        # if preds >= 0.5, we will round up to 1
        # if preds < 0.5, we will round down to 0
        if(preds[i] > 0.5):
            pred_class.append(1)
        else:
            pred_class.append(0)
    
    return np.array(pred_class)

In [4]:
    #Getting the dataframe from the csv
    df = pd.read_csv("/Users/rohansingh/Documents/CSDS 391/AI_Code/Machine Learning/Classification/irisdata.csv")
    
    #Extracting the 2nd and 3rd classes from the data frame
    df = df[df["species"] != "setosa"]
    vals = df.iloc[:,2:4]
    target = df.iloc[:,-1]
    
    vals = vals.to_numpy()
    target = target.to_numpy()
    
    #Changing species from names to binary values 
    temp = []
    for i in range (0,len(target),1):
        if(target[i] == "versicolor"):
            temp.append(0)
        else:
            temp.append(1)
    target = np.array(temp)

In [5]:
#Function to calculate the mean squareed error
def calculate_MSE(data,weights,bias,target):
    
    #Predicting using the neural network predict function
    y = predict(data,weights,bias)
    mse = 0
    
    #Iterating through all of the data points
    for i in range(0,len(y),1):
        #Adding the sum of the differences
        mse += ((y[i] - target[i])**2)
    
    #Dividing the sum by 2
    mse /= 2
    
    return mse


In [7]:
vals[0][0]

4.7

In [20]:
#Function that calculates the summed gradient
def get_summed_gradient_2(w,data,target):
    gradient = []
    bias_gradient = 0
    
    #For the bias gradient we just have to sum up all of the differences in the prediction and target
    for i in range(0,len(data),1):
        temp = np.dot(w,data[i])
        temp -= target[i]
        bias_gradient += temp
    
    #Iterating through all of the weights
    for i in range(0,len(w),1):
        grad_sum = 0
        
        #Iterating through all of the data points
        for j in range(0,len(data),1):
            temp = np.dot(w,data[j])
            temp -= target[j]
            temp *= data[j][i]
            grad_sum += temp
        
        gradient.append(grad_sum)
    
    return np.array(gradient),bias_gradient
    

In [22]:
gradients, bias_grad = get_summed_gradient(np.array([-0.054,0.495]),vals,target)
bias_grad

6.46959999999999

In [19]:
calculate_MSE(vals,np.array([-0.054,0.495]),0,target)

25.0

In [46]:
#Function that calculates the summed gradient
def get_summed_gradient(w,b,data,target):
    gradient = []
    bias_gradient = 0
    
    _class = predict(data,w,b)
    
    #For the bias gradient we just have to sum up all of the differences in the prediction and target
    for i in range(0,len(data),1): 
        bias_gradient += _class[i] - target[i]
    
    #Iterating through all of the weights
    for i in range(0,len(w),1):
        grad_sum = 0
        
        #Iterating through all of the data points
        for j in range(0,len(data),1):
            temp = _class[j] - target[j]
            temp *= data[j][i]
            grad_sum += temp
        
        gradient.append(grad_sum)
    
    return np.array(gradient),bias_gradient

In [26]:
gradients, bias_grad = get_summed_gradient_2(np.array([-0.054,0.495]),0,vals,target)
print(gradients)
print(bias_grad)

[213.   66.3]
50


In [28]:
gradients, bias_grad = get_summed_gradient_2(np.array([0,0]),0,vals,target)
print(gradients)
print(bias_grad)

[-277.6 -101.3]
-50


In [34]:
gradients, bias_grad = get_summed_gradient_2(np.array([-0.05,0.51]),-0.6,vals,target)
print(gradients)
print(bias_grad)

[-11.7  -2.5]
-2


In [38]:
gradients, bias_grad = get_summed_gradient(np.array([-0.05,0.51]),-0.6,vals,target)
print(gradients)
print(bias_grad)

[-11.7  -2.5]
-2


In [39]:
#This method will plot the linear decision boundary
def get_Linear_db(df,w,b,_class):
    #Plotting the decision boundary
    
    #Getting the inctercept and the slope
    c = -(b/w[1])
    m = -(w[0]/w[1])
    
    #Making the arrays for the x and y axes
    xd = np.array([2.5,7])
    yd = m*xd + c
    
    fig_1 = go.Figure()
    fig_1.add_trace(
                go.Scatter(x=df["petal_length"], y=df["petal_width"],
                mode='markers',
                name='points',
                marker = {'color':_class}
                ))
    fig_1.add_trace(go.Scatter(x=xd, y=yd,
                mode='lines',
                name='decision boundary'))
    fig_1.show()

In [40]:
#Function to see the change in the decision boundary
def change_in_db(df,init_w,init_b,data,target,step_size,num_iter):
    #Initializing the weights and bias
    w = init_w
    b = init_b
    
    for i in range(0,num_iter,1):
        #Getting the predicted class
        pred = predict(data,w,b)
        
        #Calling the helper function to plot the linear decision boundary
        get_Linear_db(df,w,b,pred)
        
        #Getting the gradients
        weight_grad, bias_grad = get_summed_gradient(w,b,data,target)
        
        #Updating the weights and bias
        w -= step_size*weight_grad
        b -= step_size*bias_grad

In [51]:
change_in_db(df,np.array([0.0,0.5]),0,vals,target,0.01,10)

/var/folders/yb/n2ch33ws4yzf1nn686jzrjs40000gn/T/ipykernel_95438/1135872304.py:6: RuntimeWarning:

divide by zero encountered in double_scalars

/var/folders/yb/n2ch33ws4yzf1nn686jzrjs40000gn/T/ipykernel_95438/1135872304.py:7: RuntimeWarning:

divide by zero encountered in double_scalars

